<a href="https://colab.research.google.com/github/WhalepigJun/contest/blob/main/%ED%94%84%EB%B0%8D%EC%96%B8%EA%B3%BC%EC%A0%9C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#프로그래밍언어론 과제3

20164834 박준근

python으로 작성하였습니다.

python으로 바로 생각하지 못한 문제들은 Haskell로 구현 후에 python으로 다시 생각해보았습니다.

In [1]:
def o(*procs):
  if len(procs)==0:
    return lambda x: x
  f = procs[0]
  g = o(*procs[1:])
  return lambda x: f(g(x))

def mapper(f): #map
  return lambda x: [f(xi) for xi in x]

def picker(p): #filter
  return lambda x: [xi for xi in x if p(xi)]

def folder(op, init):
  def reduce(x):
    if len(x)==0:
      return init
    return op(x[0],reduce(x[1:]))
  return reduce

import operator as op

def C(f,x): return lambda y: f(x,y)
def R(f,x): return lambda y: f(y,x)
def W(oper): return lambda x: oper(x,x)

double = W(op.add)
square = W(op.mul)

print(o(R(op.sub,1), W(op.mul), C(op.add,1), C(op.mul,2))(4))

adder = folder(op.add,0)

def iota(n):
  return range(1, n+1)
print(o(adder,iota)(10))

sigma = o(adder, iota)
factorial = o(folder(op.mul,1),iota)
print(o(mapper(sigma), iota)(10))
print(o(mapper(factorial), iota)(10))

def quicksort(pred):
  def sort(lst):
    if len(lst)==0:
      return lst
    pivot = lst[0]
    satisfied = picker(R(pred, pivot))(lst[1:])
    unsatisfied = picker(o(op.not_, R(pred, pivot)))(lst[1:])
    return sort(satisfied) + [pivot] + sort(unsatisfied)
  return sort
print(quicksort(op.gt)([5,1,2,5,-1,2,3,5,-2,-3]))

def insertionSort(pred):
  def insert(item,sorted):
    if len(sorted)==0:
      return [item]
    elif pred(item, sorted[0]):
      return [item]+sorted
    return [sorted[0]] + insert(item, sorted[1:])
  return folder(insert,[])
print(insertionSort(op.gt)([5,1,2,5,-1,2,3,5,-2,-3]))

def const(c):
  return lambda x: c

guessWhat = o(adder, mapper(const(1)))
print(guessWhat(["a","b","c","d","e"]))

def takeWhile(p):
  return folder(lambda x, acc: [x]+acc if p(x) else [], [])
print(takeWhile(o(R(op.lt, 200), W(op.mul)))(iota(20)))

def primes(m):
  def sieve(lst):
    if len(lst)==0:
      return lst
    return [lst[0]]+sieve(
        picker(o(R(op.gt,0), R(op.mod, lst[0])))(lst[1:]))
  return sieve(list(range(2,m+1)))
print(primes(10))

80
55
[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]
[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]
[5, 5, 5, 3, 2, 2, 1, -1, -2, -3]
[5, 5, 5, 3, 2, 2, 1, -1, -2, -3]
5
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[2, 3, 5, 7]




---



---



---



---



---



---



---

In [ ]:
# 필요한 함수
def o(*procs):
  if len(procs)==0:
    return lambda x: x
  f = procs[0]
  g = o(*procs[1:])
  return lambda x: f(g(x))

def mapper(f): #map
  return lambda x: [f(xi) for xi in x]

def picker(p): #filter
  return lambda x: [xi for xi in x if p(xi)]

def folder(op, init):
  def reduce(x):
    if len(x)==0:
      return init
    return op(x[0],reduce(x[1:]))
  return reduce

import operator as op

def C(f,x): return lambda y: f(x,y)
def R(f,x): return lambda y: f(y,x)
def W(oper): return lambda x: oper(x,x)

adder = folder(op.add,0)

def iota(n):
  return range(1, n+1)

def const(c):
  return lambda x: c

1.

In [2]:
# 1-1 count
print(o(adder, mapper(const(1)), picker(C(op.eq,"a")))(["1","b","a",["c","a"]]))

def count(x):
  return o(adder, mapper(const(1)), picker(C(op.eq, x))) 
count("a")(["1","b","a",["c","a"]])

1


1

In [3]:
def deep_picker(p):
  return lambda x: [xii for xi in x for xii in xi if p(xii)]

In [4]:
# 1-2 countall
print(o(adder, mapper(const(1)), deep_picker(C(op.eq,"a")))(["1","b","a",["c","a"]]))

def countall(x):
  return o(adder, mapper(const(1)), deep_picker(C(op.eq,x)))
countall("a")(["1","b","a",["c","a"]])

2


2

2.

In [5]:
# reverse_folder1
def reverse_folder1(op, init):
  def reduce(x):
    if len(x)==0:
      return init
    return op([x[len(x)-1]], reduce(x[:len(x)-1])) # 원래는 리스트의 인덱스를 끝에서부터 더해지도록 변경
  return reduce

In [6]:
# reverse1
print(reverse_folder1(op.add,[])(["a","b",["c","d"],"e"]))

reverse_adder1 = reverse_folder1(op.add,[])
reverse_adder1(["a","b",["c","d"],"e"])

['e', ['c', 'd'], 'b', 'a']


['e', ['c', 'd'], 'b', 'a']

In [7]:
# reverse_folder2
def reverse_folder2(op, init):
  def reduce(x):
    if len(x)==0:
      return init
    return R(op, [x[0]])(reduce(x[1:])) # R(f,x)를 이용해 끝에서부터 더해지도록 설정
  return reduce

In [8]:
# reverse2
print(reverse_folder2(op.add,[])(["a","b",["c","d"],"e"]))

reverse_adder2 = reverse_folder2(op.add,[])
reverse_adder2(["a","b",["c","d"],"e"])

['e', ['c', 'd'], 'b', 'a']


['e', ['c', 'd'], 'b', 'a']

In [9]:
# twist_folder1
def twist_folder1(op, init):
  def reduce(x):
    if len(x)==0:
      return init
    if len(x)==1:
      return x
    return op([reduce(x[len(x)-1])], reduce(x[:len(x)-1]))
  return reduce

In [10]:
# twist1
print(twist_folder1(op.add,[])(["a","b",["c","d"],"e"]))

twist_adder1 = twist_folder1(op.add,[])
twist_adder1(["a","b",["c","d"],"e"])

['e', ['d', 'c'], 'b', 'a']


['e', ['d', 'c'], 'b', 'a']



---

In [11]:
# twist_folder2
def twist_folder2(op, init):
  def reduce(x):
    if len(x)==0:
      return init
    if len(x)==1:
      return x
    return R(op, [reduce(x[0])])(reduce(x[1:]))
  return reduce

In [12]:
# twist2
print(twist_folder2(op.add,[])(["a","b",["c","d"],"e"]))

twist_adder2 = twist_folder2(op.add,[])
twist_adder2(["a","b",["c","d"],"e"])

['e', ['d', 'c'], 'b', 'a']


['e', ['d', 'c'], 'b', 'a']

###3.

In [13]:
def insertionSort(pred):
  def insert(item,sorted):
    if len(sorted)==0:
      return [item]
    elif pred(item, sorted[0]):
      return [item]+sorted
    return [sorted[0]] + insert(item, sorted[1:])
  return folder(insert,[])

In [14]:
insertionSort(op.le)([4,3,2,6,8,5])

[2, 3, 4, 5, 6, 8]



---



---



---



---



---



---



---

###bonus problems

#### permutations

In [15]:
#### Haskell
### ways of splitting a list into two parts
# splits :: [a] -> [([a],[a])]
# splits [] = [([],[])]
# splits (h:at) = ([],h:at) : [(h:bt,ct) | (bt,ct) <- splits at]

### perms of a list
# perms :: [a] -> [[a]]
# perms [] = [[]]
# perms (h:at) = [bt ++ h:ct | perm <- perms at, (bt,ct) <- splits perm]

# perms [1,2,3]
#   --> [[1,2,3],[2,1,3],[2,3,1],[1,3,2],[3,1,2],[3,2,1]]

# 함수형언어 Haskell로 구현하고, python으로 옮겨보았습니다.

In [16]:
def split(i):
  def partition(lst):
    if len(lst)==0:
      return lst
    return (lst[:i],lst[i:])
  return partition

def splits(lst):
  return [split(x)(lst) for x in range(len(lst)+1)]

def perms(lst):
  if len(lst)==1:
    return [lst]
  return [x[0]+[lst[0]]+x[1] for perm in perms(lst[1:]) for x in splits(perm)]

In [17]:
perms([1,2,3])

[[1, 2, 3], [2, 1, 3], [2, 3, 1], [1, 3, 2], [3, 1, 2], [3, 2, 1]]

In [18]:
perms([1,2,3,4])

[[1, 2, 3, 4],
 [2, 1, 3, 4],
 [2, 3, 1, 4],
 [2, 3, 4, 1],
 [1, 3, 2, 4],
 [3, 1, 2, 4],
 [3, 2, 1, 4],
 [3, 2, 4, 1],
 [1, 3, 4, 2],
 [3, 1, 4, 2],
 [3, 4, 1, 2],
 [3, 4, 2, 1],
 [1, 2, 4, 3],
 [2, 1, 4, 3],
 [2, 4, 1, 3],
 [2, 4, 3, 1],
 [1, 4, 2, 3],
 [4, 1, 2, 3],
 [4, 2, 1, 3],
 [4, 2, 3, 1],
 [1, 4, 3, 2],
 [4, 1, 3, 2],
 [4, 3, 1, 2],
 [4, 3, 2, 1]]

In [19]:
# permutation의 개수
print(o(adder, mapper(const(1)), perms)([1,2,3,4]),"개")

number_of_perms = o(adder, mapper(const(1)), perms)
number_of_perms([1,2,3,4])

24 개


24

#### powerset

In [20]:
# splits 함수를 약간 변형

def split_(i):
  def partition(lst):
    if len(lst)==0:
      return lst
    return [lst[:i]]+[lst[i:]]
  return partition

def splits_(lst):
  return [split_(x)(lst) for x in range(1,len(lst))]

In [21]:
o(folder(op.add, []), splits_)([1,2,3])

[[1], [2, 3], [1, 2], [3]]

In [22]:
def powerset(lst):
  if len(lst)==0 or len(lst)==1:
    return [lst]
  return [[]] + o(folder(op.add,[]), splits_)(lst) + powerset(lst[1:len(lst)-1])

In [23]:
powerset([1,2,3])

[[], [1], [2, 3], [1, 2], [3], [2]]

In [24]:
powerset([1,2,3,4,5])

[[],
 [1],
 [2, 3, 4, 5],
 [1, 2],
 [3, 4, 5],
 [1, 2, 3],
 [4, 5],
 [1, 2, 3, 4],
 [5],
 [],
 [2],
 [3, 4],
 [2, 3],
 [4],
 [3]]

In [25]:
# powerset의 개수
print(o(adder, mapper(const(1)), powerset)([1,2,3,4,5]), "개")

number_of_powerset = o(adder, mapper(const(1)), powerset)
number_of_powerset([1,2,3,4,5])


15 개


15